In [1]:
from nltk.corpus import reuters
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
fids_train = [fid for fid in reuters.fileids() if fid.startswith('train')]
fids_test = [fid for fid in reuters.fileids() if fid.startswith('test')]

tfidf = TfidfVectorizer().fit(reuters.raw(fid) for fid in fids_train)
X_train = tfidf.transform(reuters.raw(fid) for fid in fids_train)
X_test = tfidf.transform(reuters.raw(fid) for fid in fids_test)

mlb = MultiLabelBinarizer().fit(reuters.categories(fid) for fid in fids_train)
y_train = mlb.transform(reuters.categories(fid) for fid in fids_train)
y_test = mlb.transform(reuters.categories(fid) for fid in fids_test)

In [3]:
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(mlb.classes_.shape[0], activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6728704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 90)                23130     
Total params: 6,817,626
Trainable params: 6,817,626
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
model.fit(X_train, y_train, shuffle=True, batch_size=32, epochs=10, validation_split=0.1)
y_test_pred = model.predict(X_test, verbose=1)

Train on 6992 samples, validate on 777 samples
Epoch 1/10
6992/6992 [==============================] - 46s 7ms/step - loss: 0.1079 - acc: 0.9782 - val_loss: 0.0338 - val_acc: 0.9920
Epoch 2/10
6992/6992 [==============================] - 46s 7ms/step - loss: 0.0320 - acc: 0.9920 - val_loss: 0.0243 - val_acc: 0.9934
Epoch 3/10
6992/6992 [==============================] - 44s 6ms/step - loss: 0.0231 - acc: 0.9936 - val_loss: 0.0182 - val_acc: 0.9948
Epoch 4/10
6992/6992 [==============================] - 42s 6ms/step - loss: 0.0176 - acc: 0.9948 - val_loss: 0.0154 - val_acc: 0.9953
Epoch 5/10
6992/6992 [==============================] - 44s 6ms/step - loss: 0.0141 - acc: 0.9956 - val_loss: 0.0138 - val_acc: 0.9958
Epoch 6/10
6992/6992 [==============================] - 43s 6ms/step - loss: 0.0115 - acc: 0.9963 - val_loss: 0.0124 - val_acc: 0.9961
Epoch 7/10
6992/6992 [==============================] - 43s 6ms/step - loss: 0.0095 - acc: 0.9969 - val_loss: 0.0116 - val_acc: 0.9965
Epoch 8/

In [5]:
# Summary interpretation (i.e. for gold):
# - precision: 95% of cases labeled gold are actually about gold
# - recall: 70% of the gold labeled (actual) articles where classified as gold
print(classification_report(y_test, y_test_pred>0.5, target_names=mlb.classes_))

                 precision    recall  f1-score   support

            acq       0.96      0.97      0.97       719
           alum       1.00      0.13      0.23        23
         barley       1.00      0.43      0.60        14
            bop       0.94      0.50      0.65        30
        carcass       1.00      0.11      0.20        18
     castor-oil       0.00      0.00      0.00         1
          cocoa       1.00      0.72      0.84        18
        coconut       0.00      0.00      0.00         2
    coconut-oil       0.00      0.00      0.00         3
         coffee       0.96      0.93      0.95        28
         copper       0.91      0.56      0.69        18
     copra-cake       0.00      0.00      0.00         1
           corn       0.83      0.79      0.81        56
         cotton       1.00      0.10      0.18        20
     cotton-oil       0.00      0.00      0.00         2
            cpi       0.86      0.43      0.57        28
            cpu       0.00    

/Users/tdiggelm/checkout/total_nn/.py/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/tdiggelm/checkout/total_nn/.py/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
